<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#functions" data-toc-modified-id="functions-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>functions</a></span></li></ul></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#exploring-&quot;SCIENCE_DATA&quot;" data-toc-modified-id="exploring-&quot;SCIENCE_DATA&quot;-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>exploring "SCIENCE_DATA"</a></span><ul class="toc-item"><li><span><a href="#right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly." data-toc-modified-id="right,-i-forgot-there's-a-readme-explaining-everything-quite-clearly.-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>right, i forgot there's a readme explaining everything quite clearly.</a></span></li><li><span><a href="#so-how-do-i-interpret-the-320x36?" data-toc-modified-id="so-how-do-i-interpret-the-320x36?-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>so how do i interpret the 320x36?</a></span><ul class="toc-item"><li><span><a href="#before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries" data-toc-modified-id="before-i-forget,-there's-probably-a-library-to-parse-lat/long-into-geographic-regions/cities/countries-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries</a></span></li></ul></li></ul></li><li><span><a href="#Data-format-is-based-off-of-instrument-format" data-toc-modified-id="Data-format-is-based-off-of-instrument-format-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data format is based off of instrument format</a></span></li><li><span><a href="#filename-considerations" data-toc-modified-id="filename-considerations-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>filename considerations</a></span></li><li><span><a href="#right,-i-think-i've-got-the-format" data-toc-modified-id="right,-i-think-i've-got-the-format-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>right, i think i've got the format</a></span><ul class="toc-item"><li><span><a href="#Matrix-matching" data-toc-modified-id="Matrix-matching-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Matrix-matching</a></span></li></ul></li><li><span><a href="#speed-tests" data-toc-modified-id="speed-tests-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>speed tests</a></span></li><li><span><a href="#append-time:-81s-/-.h5" data-toc-modified-id="append-time:-81s-/-.h5-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>append time: 81s / .h5</a></span></li><li><span><a href="#TIME-issue:" data-toc-modified-id="TIME-issue:-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>TIME issue:</a></span><ul class="toc-item"><li><span><a href="#matrix-to-DataFrame-directly" data-toc-modified-id="matrix-to-DataFrame-directly-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>matrix to DataFrame directly</a></span></li><li><span><a href="#nice.-i-can-just-flatten-the-arrays-and-dataframe-it" data-toc-modified-id="nice.-i-can-just-flatten-the-arrays-and-dataframe-it-1.7.2"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>nice. i can just flatten the arrays and dataframe it</a></span></li></ul></li><li><span><a href="#matrix-test:-much-faster,-but-warning" data-toc-modified-id="matrix-test:-much-faster,-but-warning-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>matrix test: much faster, but warning</a></span><ul class="toc-item"><li><span><a href="#now-build-the-process_all_orbits()" data-toc-modified-id="now-build-the-process_all_orbits()-1.8.1"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>now build the process_all_orbits()</a></span></li></ul></li></ul></li></ul></div>

### imports

In [44]:
import h5py
import os
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import time

### functions

In [154]:
def peek_dataset(ds): # only works on 2d array datasets
    for i in range(ds.shape[0]):
        for k in range(ds.shape[1]):
            print(ds[i][k])
        print('\n')
        
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'sat_alt':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df
    

def process_orbit_old(df, h5): # pandas DF, string. adds h5's observations to dataframe
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...')
            obs = {} # 11,520 max
            obs['lat'] = geo['Latitude'][i][k]
            obs['long'] = geo['Longitude'][i][k]
            obs['sat_lat'] = geo['SpacecraftLatitude'][i] # 1d so just use 1st loop
            obs['sat_long'] = geo['SpacecraftLongitude'][i]
            obs['time'] = geo['TimeUTC'][i]
            obs['sza'] = geo['SolarZenithAngle'][i]
            obs['pbl'] = sci['ColumnAmountSO2_PBL'][i][k]
            obs['anom'] = sci['Flag_SAA'][i][k]
            obs['cloud'] = sci['RadiativeCloudFraction'][i][k]
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            df = df.append(obs, ignore_index=True)
            c+=1
    # end loops
    print(f'dataframe length: {len(df)}')
    return df

# EDA

In [2]:
filename = "many_outs.h5"

okay, found the issue
- for downloading multiple .h5 files, nasa reccomends:
    - cat <url.txt> | tr -d '\r' | xargs -n 1 curl -LJO -n -c ~/.urs_cookies -b ~/.urs_cookies
    - but i was getting some access denied + use --output recommendations. didn't work
        - so i tried >> piping it to a file. i could have named it "data.h5", maybe? can you store h5s in other h5s?
        - i named it 'data.iso' on someone's SO solution
        - this went poorly. the data downloaded, but when trying to open it, it seemed super corrupted, even with specific encoding that allowed it to open
    - i went back to NASA's downloading instructions and tested the curl cmd with a smaller subset of .h5 files, which worked (i think i had some typos in the cmdline, or i was formatting incorrectly) so i applied it to the original 2012-2020 46gb of data. currently downloading all the .h5s to my home directory, so i'll just have to move them en masse and loop-open them into a dataframe in the project folder (in 5 hours when it downloads).

In [3]:
# import os
# for filename in os.listdir(os.getcwd()):
#    with open(os.path.join(os.cwd(), filename), 'r') as f: # open in readonly mode
#       # do your stuff

In [4]:
lst = []
for filename in os.listdir(os.getcwd()):
    if filename.endswith('.h5'):
        with h5py.File(filename, 'r') as f:
            print('keys: %s' %f.keys())
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])

keys: <KeysViewHDF5 ['ANCILLARY_DATA', 'GEOLOCATION_DATA', 'SCIENCE_DATA', 'nTimes', 'nWavel2', 'nWavel3', 'nXtrack']>


In [5]:
import h5py
f = h5py.File('one.h5', 'r')

In [6]:
f.keys()

<KeysViewHDF5 ['ANCILLARY_DATA', 'GEOLOCATION_DATA', 'SCIENCE_DATA', 'nTimes', 'nWavel2', 'nWavel3', 'nXtrack']>

In [7]:
for k in f.keys():
    print(k, type(f[k]), '\n') # remember to use the key on the dict obj to actually get the type of value

ANCILLARY_DATA <class 'h5py._hl.group.Group'> 

GEOLOCATION_DATA <class 'h5py._hl.group.Group'> 

SCIENCE_DATA <class 'h5py._hl.group.Group'> 

nTimes <class 'h5py._hl.dataset.Dataset'> 

nWavel2 <class 'h5py._hl.dataset.Dataset'> 

nWavel3 <class 'h5py._hl.dataset.Dataset'> 

nXtrack <class 'h5py._hl.dataset.Dataset'> 



- so the root h5py object is a dictionary... of sorts.
    - we've got Groups, which are like folders, and Datasets, which are like arrays (but different)

In [8]:
f['GEOLOCATION_DATA'].keys() # looks like location of Suomi satellite at moment of reading
# we're gonna want Latitude and Longitude to identify 'where' the reading is.
# maybe TimeUTC as well, that could save a lot of re-coding work for broad regional targeting

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [9]:
f['ANCILLARY_DATA'].keys()

<KeysViewHDF5 ['CloudPressure', 'TerrainPressure']>

In [10]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

from the NASA dataset description:

The goal of this dataset is to create and archive a Level 2 Sulfur Dioxide (SO2) Earth Science Data Record (ESDR) from backscatter Ultraviolet (BUV) measurements from the Ozone Mapping and Profiler Suite (OMPS) Nadir Mapper (NM) flown on the NASA/NOAA Suomi National Polar-orbiting Partnership (S-NPP) satellite since 2011. We apply the NASA Goddard Space Flight Center (GSFC) principal component analysis (PCA) satellite trace gas retrieval algorithm, to obtain the best measurement-based ESDR of volcanic and anthropogenic SO2 loadings and emissions, which is fully consistent with our SO2 ESDR (OMSO2) from the Ozone Monitoring Instrument (OMI) flown on the NASA Earth Observing System (EOS) Aura satellite since 2004. 

The SO2 ESDR released here at the Goddard Earth Science (GES) Data and Information Data Center (DISC), OMPS_NPP_NMSO2_PCA_L2, is a stand-alone dataset, but will also be part of the NASA's Making Earth System Data Records for Use in Research Environments (MEaSUREs) program. It covers the entire period of the S-NPP/OMPS mission since launch in October 2011, and forward data processing is ongoing. OMPS_NPP_NMSO2_PCA_L2 is a Level 2 orbital swath product, which will be used to study SO2 pollutants emitted from large point sources such as coal-fired power plants and smelters as well as to monitor volcanic SO2 emissions.

Sulfur Dioxide (SO2) is a short-lived gas primarily produced by volcanoes, power plants, refineries, metal smelting and burning of fossil fuels. Where SO2 remains near the Earth's surface, it is toxic, causes acid rain, and degrades air quality. Where SO2 is lofted into the free troposphere, it forms aerosols that can alter cloud reflectivity and precipitation. In the stratosphere, volcanic SO2 forms sulfate aerosols that can result in climate change.

- translation:
    - here's SO2 levels by latitude, date & many other variables
        - derived from applying PCA algos to Ozone mapping data, which was measured from backscatter ultraviolet readings
                - basically, NASA looks at backscattered UV off of the atmosphere - off of ozone, but can effectively derive SO2 from the ozone reading using PCA
        - seems convoluted, but it's consistent with their last SO2 reading system from 2004, and it's probably the best estimate the world's got right now (unless anyone would like to step up and accurately measure trace gas concentrations globally every day. seems bloody hard in my opinion)

## exploring "SCIENCE_DATA"

In [11]:
f['SCIENCE_DATA'].keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

In [12]:
f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

<HDF5 dataset "ColumnAmountSO2_PBL": shape (320, 36), type "<f4">

In [13]:
pbl = f['SCIENCE_DATA']['ColumnAmountSO2_PBL']

In [58]:
geo.keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [63]:
geo['TimeUTC'][310]

b'2012-06-05T23:36:57.253489Z'

In [15]:
# peek_dataset(pbl) # a 320x36 matrix. now to figure out why there's 11520 readings in this dataset, which is one .h5 file


In [16]:
for k in f['SCIENCE_DATA'].keys(): # are they all 320x36? probably the SO2 at least
#     print(k, type(k))
    if 'SO2' in k: # k is a string
        print(k, f['SCIENCE_DATA'][k].shape) # yes, the SO2 readings are matched in size.
# probably different instruments or estimation methods. perhaps we could aggregate them later on, but they're all going in the dataframe for now

ColumnAmountSO2_PBL (320, 36)
ColumnAmountSO2_STL (320, 36)
ColumnAmountSO2_TRL (320, 36)
ColumnAmountSO2_TRM (320, 36)
ColumnAmountSO2_TRU (320, 36)
Flag_SO2 (320, 36)


### right, i forgot there's a readme explaining everything quite clearly.
- units
    - 5 estimates (principal components), related to various geophysical processes (ozone absorption, rotational Raman scattering) and instrument measurement details (wavelength shift, dark current)
    - PCs used in SO2 spectral fitting to reduce interferences
        - so we've got five estimates of total "SO2 Vertical Column Density" in Dobson Units (1DU = 2.69 * 10^16 molecules/cm^2)
            - note the "cm^2". these are flat "slices" of a column? no
            - each of the 5 estimates corresponds to a different assumed SO2 cloud height, or Center of Mass Altitude (CMA)
            - all 5 VCD values represent the estimated total SO2 burden within entire atmospheric column:
                - should not be interpreted as partial column amounts within different layers/parts of atmosphere
                - they want users to select 1 or interpolate between two estimates that are most representative of conditions for a particular case of interest
- so each of the 5 estimates measures with different heights/sensitivities, and therefore are 'tuned' to measure different... types of SO2 emission.
    - anthropogenic (coal factories etc) SO2 emissions will look different, and be measured differently from volcanic eruptions
    - that sort of thing

- PBL: Planetary Boundary Layer
    - assumes that SO2 is predominantly in lowest 1km of atmosphere
    - use for studies on near-surface pollution
        - for each OMPS orbit:
            - process 36 rows (cross-track positions) one at a time
            - use PCA to extract PCs for spectral range 310.5-340nm from sun-normalized BUV radiance spectra
            - PCs are ranked in descending order according to spectral variance that they explain
    - due to jacobian simplication, they recommend only using snow/ice free pixels with small radiative cloud fraction
- the other 4 column amounts seem to be volcanic
### so how do i interpret the 320x36?
- wait, they're talking about pixels
    - 320x36 is the "resolution" of the measurement
    
#### before i forget, there's probably a library to parse lat/long into geographic regions/cities/countries

- it's an "orbital-track" product about the SNPP satellite.
    - polar sun-synchronous orbit
## Data format is based off of instrument format
- OMPS-NM has 110° FoV, cross-track swath ~= 2800km
    - global coverage,14-15 orbits/day
    - "nominal spatial resolution" is 50km x 50km at nadir in nominal observation mode
        - there's a higher res mode recorded once a week, but that's not in this dataset
- for each OMPS orbit, process 36 rows (cross-track positions) use PCA to extract PCs for spectral range from BUV
    - so, 320 PCs per position while crossing orbital track?
        - what does the lat/long data look like? that might solve this question


In [17]:
lat = f['GEOLOCATION_DATA']['Latitude']
long = f['GEOLOCATION_DATA']['Longitude']
print(lat.shape, long.shape)

(320, 36) (320, 36)


- so in one .h5's 36x320:
    - began at -29, -152: about 1/3 of the way from NZ to Ecuador
    - ended at 67, 16: northern middle Norway
        - but those aren't the extreme bounds for lat or long.
            - without plotting it out, my intuition says it's an oscillating path

In [18]:
# peek_dataset(lat)
# peek_dataset(long)

- now the 50km spatial resolution makes sense.
    - we've got 11520 'pixels' or 'granules' (are granules the .h5?)
    - each pixel has an SO2 PBL reading, as well as a lat/long
        - so the corresponding [x],[y] position in the matrix will match a geolocation to its PBL
- warnings from the readme:
    - all pixels:
        - with large solar zenith angle (SZA>70)
        - near edge of swath (rows 1-2, 35-36)
        - affected by South Atlantic Anomaly (flag_SAA=1)
            - should be excluded
    - PBL SO2:
        - only use:
            - snow/ice free pixels
            - with RadiativeCloudFraction (<0.3)
            
    

## filename considerations
- each .h5 file contains:
    - date/time at start of orbit
    - orbit number
        - so each .h5 is an orbit. question answered
- what about time? that should help me figure out "unique positions"

In [19]:
geo = f['GEOLOCATION_DATA']
time = geo['TimeUTC']
time.shape # ah

(320,)

In [20]:
geo.keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'SolarAzimuthAngle', 'SolarZenithAngle', 'SpacecraftAltitude', 'SpacecraftLatitude', 'SpacecraftLongitude', 'TimeTAI93', 'TimeUTC', 'ViewingAzimuthAngle', 'ViewingZenithAngle']>

In [21]:
lat = geo['Latitude']
lat.shape

(320, 36)

In [22]:
lat[0][5]

-27.383163

- so we've got 320 times per orbit, but 320x36 pixels and 320x36 lat/long coords
- that means at 320 times throughout the day, the satellite gathers 36 pixels, and each one has a unique geolocation...?
    - the 36 is "cross-track positions"
        - is it a vertical "36 slices of column underneath spacecraft"?
            - no
            - if that were the case, 
                - there wouldn't be a separate lat/long for spacecraft as well
                    - the "spacecraft lat/long" is (320) while "lat/long" is (320,36)
                    - lines up with 320 timestamps
    - how to interpret "cross-track positions"
        - 36 positions that cross the track (orbital track path over earth's surface)
        - so for each 320 position+timestamp along track, there's 36 "positions" that cross the track
        - i'm assuming it does a left/right sweep below in its 110° fov while continuing "forwards"
            - this makes the most sense. spacecraft geocoords are 1d, geocoords are 2d, so the 2nd dimension from line to plane has to be added somehow
            - i could be stupendously wrong, but let's worry about that later

## right, i think i've got the format
- each .h5 file is an orbit, with 320 track positions (geocoords+timestamp)
    - each track position has 36 cross-track positions, so we get 320x36 matrix for readings+GPS
    - each track/crosstrack combo is a 50km x 50km space of earth (i think...)
        - the lat/long is decimals, but that can be converted to degrees if needed
- for each .h5 file, we've got 11,520 PBL readings, but not all of them will be usable
    - ice/snow, large solar zenith angle, south atlantic magnetic interference
- and we've got something like... 40,000 or 50,000 h5s, maybe more
### Matrix-matching
- lat[5][6], long[5][6] and PBT[5][6] will give us the location and reading for the 5th position, 6th cross position
    - so we can just write functions to neatly iterate through each h5's file tree and add each 'observation' to a big dataframe
    - it's going to be big


## speed tests
- append or concat? 11,520 matrix elements, 36,000ish orbits

In [ ]:


start = time.time()
print("hello")
end = time.time()
print(end - start)

In [55]:
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df

def process_orbit_app(df, h5): # pandas DF, string. adds h5's observations to dataframe
    start = time.time()
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...')
            obs = {} # 11,520 max
            obs['lat'] = geo['Latitude'][i][k]
            obs['long'] = geo['Longitude'][i][k]
            obs['sat_lat'] = geo['SpacecraftLatitude'][i] # 1d so just use 1st loop
            obs['sat_long'] = geo['SpacecraftLongitude'][i]
            obs['time'] = geo['TimeUTC'][i]
            obs['sza'] = geo['SolarZenithAngle'][i]
            obs['pbl'] = sci['ColumnAmountSO2_PBL'][i][k]
            obs['anom'] = sci['Flag_SAA'][i][k]
            obs['cloud'] = sci['RadiativeCloudFraction'][i][k]
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            df = df.append(obs, ignore_index=True)
            c+=1
    # end loops
    print(f'dataframe length: {len(df)}')
    end = time.time()
    print(f'TIME .append()  :  {end - start}')
    return df

def process_orbit_list(df, h5): # pandas DF, string. adds h5's observations to dataframe
#     start = time.time()
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    c=0
    lat, long, sat_lat, sat_long, time, sza, pbl, anom, cloud = ([] for i in range(9)) # create 9 empty lists
    for i in range(320): # track positions during orbit
        for k in range(36): # cross positions at each track position
            print(f'adding {c}...') # add to each list
            lat.append(geo['Latitude'][i][k])
            long.append(geo['Longitude'][i][k])
            sat_lat.append(geo['SpacecraftLatitude'][i]) # 1d so just use 1st loop
            sat_long.append(geo['SpacecraftLongitude'][i])
            time.append(geo['TimeUTC'][i])
            sza.append(geo['SolarZenithAngle'][i])
            pbl.append(sci['ColumnAmountSO2_PBL'][i][k])
            anom.append(sci['Flag_SAA'][i][k])
            cloud.append(sci['RadiativeCloudFraction'][i][k])
            # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
            c+=1
    # end loops
    df = pd.DataFrame(list(zip(lat, long, sat_lat, sat_long, time, sza, pbl, anom, cloud)),
                     columns=["lat", "long", "sat_lat", "sat_long", "time", "sza", "pbl", "anom", "cloud"])
    print(f'dataframe length: {len(df)}')
#     end = time.time()
#     print(f'TIME .append()  :  {end - start}')
    return df

In [48]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)


In [49]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

'df3'

'df4'

"pd.concat([df3, df4], axis='col')"

In [51]:
pd.concat([df3, df4], axis=1)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


In [52]:
# list of strings 
lst = ['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks'] 
  
# list of int 
lst2 = [11, 22, 33, 44, 55, 66, 77] 
lst3 = ['a','b','c','d','e','f','g']
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
df = pd.DataFrame(list(zip(lst, lst2, lst3)), 
               columns =['Name', 'val', 'chr']) 

In [53]:
df

,Name,val,chr
0,Geeks,11,a
1,For,22,b
2,Geeks,33,c
3,is,44,d
4,portal,55,e
5,for,66,f
6,Geeks,77,g


In [57]:

filename = 'one.h5'
df = create_base_df()
start = time.time()
df = process_orbit_app(df, filename)
end = time.time()
print(f'TIME .append()  :  {end - start}')

adding 0...
adding 1...
adding 2...
adding 3...
adding 4...
adding 5...
adding 6...
adding 7...
adding 8...
adding 9...
adding 10...
adding 11...
adding 12...
adding 13...
adding 14...
adding 15...
adding 16...
adding 17...
adding 18...
adding 19...
adding 20...
adding 21...
adding 22...
adding 23...
adding 24...
adding 25...
adding 26...
adding 27...
adding 28...
adding 29...
adding 30...
adding 31...
adding 32...
adding 33...
adding 34...
adding 35...
adding 36...
adding 37...
adding 38...
adding 39...
adding 40...
adding 41...
adding 42...
adding 43...
adding 44...
adding 45...
adding 46...
adding 47...
adding 48...
adding 49...
adding 50...
adding 51...
adding 52...
adding 53...
adding 54...
adding 55...
adding 56...
adding 57...
adding 58...
adding 59...
adding 60...
adding 61...
adding 62...
adding 63...
adding 64...
adding 65...
adding 66...
adding 67...
adding 68...
adding 69...
adding 70...
adding 71...
adding 72...
adding 73...
adding 74...
adding 75...
adding 76...
adding 77

adding 614...
adding 615...
adding 616...
adding 617...
adding 618...
adding 619...
adding 620...
adding 621...
adding 622...
adding 623...
adding 624...
adding 625...
adding 626...
adding 627...
adding 628...
adding 629...
adding 630...
adding 631...
adding 632...
adding 633...
adding 634...
adding 635...
adding 636...
adding 637...
adding 638...
adding 639...
adding 640...
adding 641...
adding 642...
adding 643...
adding 644...
adding 645...
adding 646...
adding 647...
adding 648...
adding 649...
adding 650...
adding 651...
adding 652...
adding 653...
adding 654...
adding 655...
adding 656...
adding 657...
adding 658...
adding 659...
adding 660...
adding 661...
adding 662...
adding 663...
adding 664...
adding 665...
adding 666...
adding 667...
adding 668...
adding 669...
adding 670...
adding 671...
adding 672...
adding 673...
adding 674...
adding 675...
adding 676...
adding 677...
adding 678...
adding 679...
adding 680...
adding 681...
adding 682...
adding 683...
adding 684...
adding

adding 1207...
adding 1208...
adding 1209...
adding 1210...
adding 1211...
adding 1212...
adding 1213...
adding 1214...
adding 1215...
adding 1216...
adding 1217...
adding 1218...
adding 1219...
adding 1220...
adding 1221...
adding 1222...
adding 1223...
adding 1224...
adding 1225...
adding 1226...
adding 1227...
adding 1228...
adding 1229...
adding 1230...
adding 1231...
adding 1232...
adding 1233...
adding 1234...
adding 1235...
adding 1236...
adding 1237...
adding 1238...
adding 1239...
adding 1240...
adding 1241...
adding 1242...
adding 1243...
adding 1244...
adding 1245...
adding 1246...
adding 1247...
adding 1248...
adding 1249...
adding 1250...
adding 1251...
adding 1252...
adding 1253...
adding 1254...
adding 1255...
adding 1256...
adding 1257...
adding 1258...
adding 1259...
adding 1260...
adding 1261...
adding 1262...
adding 1263...
adding 1264...
adding 1265...
adding 1266...
adding 1267...
adding 1268...
adding 1269...
adding 1270...
adding 1271...
adding 1272...
adding 127

adding 1756...
adding 1757...
adding 1758...
adding 1759...
adding 1760...
adding 1761...
adding 1762...
adding 1763...
adding 1764...
adding 1765...
adding 1766...
adding 1767...
adding 1768...
adding 1769...
adding 1770...
adding 1771...
adding 1772...
adding 1773...
adding 1774...
adding 1775...
adding 1776...
adding 1777...
adding 1778...
adding 1779...
adding 1780...
adding 1781...
adding 1782...
adding 1783...
adding 1784...
adding 1785...
adding 1786...
adding 1787...
adding 1788...
adding 1789...
adding 1790...
adding 1791...
adding 1792...
adding 1793...
adding 1794...
adding 1795...
adding 1796...
adding 1797...
adding 1798...
adding 1799...
adding 1800...
adding 1801...
adding 1802...
adding 1803...
adding 1804...
adding 1805...
adding 1806...
adding 1807...
adding 1808...
adding 1809...
adding 1810...
adding 1811...
adding 1812...
adding 1813...
adding 1814...
adding 1815...
adding 1816...
adding 1817...
adding 1818...
adding 1819...
adding 1820...
adding 1821...
adding 182

adding 2324...
adding 2325...
adding 2326...
adding 2327...
adding 2328...
adding 2329...
adding 2330...
adding 2331...
adding 2332...
adding 2333...
adding 2334...
adding 2335...
adding 2336...
adding 2337...
adding 2338...
adding 2339...
adding 2340...
adding 2341...
adding 2342...
adding 2343...
adding 2344...
adding 2345...
adding 2346...
adding 2347...
adding 2348...
adding 2349...
adding 2350...
adding 2351...
adding 2352...
adding 2353...
adding 2354...
adding 2355...
adding 2356...
adding 2357...
adding 2358...
adding 2359...
adding 2360...
adding 2361...
adding 2362...
adding 2363...
adding 2364...
adding 2365...
adding 2366...
adding 2367...
adding 2368...
adding 2369...
adding 2370...
adding 2371...
adding 2372...
adding 2373...
adding 2374...
adding 2375...
adding 2376...
adding 2377...
adding 2378...
adding 2379...
adding 2380...
adding 2381...
adding 2382...
adding 2383...
adding 2384...
adding 2385...
adding 2386...
adding 2387...
adding 2388...
adding 2389...
adding 239

...

adding 9005...
adding 9006...
adding 9007...
adding 9008...
adding 9009...
adding 9010...
adding 9011...
adding 9012...
adding 9013...
adding 9014...
adding 9015...
adding 9016...
adding 9017...
adding 9018...
adding 9019...
adding 9020...
adding 9021...
adding 9022...
adding 9023...
adding 9024...
adding 9025...
adding 9026...
adding 9027...
adding 9028...
adding 9029...
adding 9030...
adding 9031...
adding 9032...
adding 9033...
adding 9034...
adding 9035...
adding 9036...
adding 9037...
adding 9038...
adding 9039...
adding 9040...
adding 9041...
adding 9042...
adding 9043...
adding 9044...
adding 9045...
adding 9046...
adding 9047...
adding 9048...
adding 9049...
adding 9050...
adding 9051...
adding 9052...
adding 9053...
adding 9054...
adding 9055...
adding 9056...
adding 9057...
adding 9058...
adding 9059...
adding 9060...
adding 9061...
adding 9062...
adding 9063...
adding 9064...
adding 9065...
adding 9066...
adding 9067...
adding 9068...
adding 9069...
adding 9070...
adding 907

adding 9575...
adding 9576...
adding 9577...
adding 9578...
adding 9579...
adding 9580...
adding 9581...
adding 9582...
adding 9583...
adding 9584...
adding 9585...
adding 9586...
adding 9587...
adding 9588...
adding 9589...
adding 9590...
adding 9591...
adding 9592...
adding 9593...
adding 9594...
adding 9595...
adding 9596...
adding 9597...
adding 9598...
adding 9599...
adding 9600...
adding 9601...
adding 9602...
adding 9603...
adding 9604...
adding 9605...
adding 9606...
adding 9607...
adding 9608...
adding 9609...
adding 9610...
adding 9611...
adding 9612...
adding 9613...
adding 9614...
adding 9615...
adding 9616...
adding 9617...
adding 9618...
adding 9619...
adding 9620...
adding 9621...
adding 9622...
adding 9623...
adding 9624...
adding 9625...
adding 9626...
adding 9627...
adding 9628...
adding 9629...
adding 9630...
adding 9631...
adding 9632...
adding 9633...
adding 9634...
adding 9635...
adding 9636...
adding 9637...
adding 9638...
adding 9639...
adding 9640...
adding 964

adding 10141...
adding 10142...
adding 10143...
adding 10144...
adding 10145...
adding 10146...
adding 10147...
adding 10148...
adding 10149...
adding 10150...
adding 10151...
adding 10152...
adding 10153...
adding 10154...
adding 10155...
adding 10156...
adding 10157...
adding 10158...
adding 10159...
adding 10160...
adding 10161...
adding 10162...
adding 10163...
adding 10164...
adding 10165...
adding 10166...
adding 10167...
adding 10168...
adding 10169...
adding 10170...
adding 10171...
adding 10172...
adding 10173...
adding 10174...
adding 10175...
adding 10176...
adding 10177...
adding 10178...
adding 10179...
adding 10180...
adding 10181...
adding 10182...
adding 10183...
adding 10184...
adding 10185...
adding 10186...
adding 10187...
adding 10188...
adding 10189...
adding 10190...
adding 10191...
adding 10192...
adding 10193...
adding 10194...
adding 10195...
adding 10196...
adding 10197...
adding 10198...
adding 10199...
adding 10200...
adding 10201...
adding 10202...
adding 1

adding 10675...
adding 10676...
adding 10677...
adding 10678...
adding 10679...
adding 10680...
adding 10681...
adding 10682...
adding 10683...
adding 10684...
adding 10685...
adding 10686...
adding 10687...
adding 10688...
adding 10689...
adding 10690...
adding 10691...
adding 10692...
adding 10693...
adding 10694...
adding 10695...
adding 10696...
adding 10697...
adding 10698...
adding 10699...
adding 10700...
adding 10701...
adding 10702...
adding 10703...
adding 10704...
adding 10705...
adding 10706...
adding 10707...
adding 10708...
adding 10709...
adding 10710...
adding 10711...
adding 10712...
adding 10713...
adding 10714...
adding 10715...
adding 10716...
adding 10717...
adding 10718...
adding 10719...
adding 10720...
adding 10721...
adding 10722...
adding 10723...
adding 10724...
adding 10725...
adding 10726...
adding 10727...
adding 10728...
adding 10729...
adding 10730...
adding 10731...
adding 10732...
adding 10733...
adding 10734...
adding 10735...
adding 10736...
adding 1

adding 11216...
adding 11217...
adding 11218...
adding 11219...
adding 11220...
adding 11221...
adding 11222...
adding 11223...
adding 11224...
adding 11225...
adding 11226...
adding 11227...
adding 11228...
adding 11229...
adding 11230...
adding 11231...
adding 11232...
adding 11233...
adding 11234...
adding 11235...
adding 11236...
adding 11237...
adding 11238...
adding 11239...
adding 11240...
adding 11241...
adding 11242...
adding 11243...
adding 11244...
adding 11245...
adding 11246...
adding 11247...
adding 11248...
adding 11249...
adding 11250...
adding 11251...
adding 11252...
adding 11253...
adding 11254...
adding 11255...
adding 11256...
adding 11257...
adding 11258...
adding 11259...
adding 11260...
adding 11261...
adding 11262...
adding 11263...
adding 11264...
adding 11265...
adding 11266...
adding 11267...
adding 11268...
adding 11269...
adding 11270...
adding 11271...
adding 11272...
adding 11273...
adding 11274...
adding 11275...
adding 11276...
adding 11277...
adding 1

## append time: 81s / .h5
## list zip time: 78s / .h5
- closer than i thought. i thought i read .concat() was faster? but then again i'm not using concat in the end - that combines dataframes

In [ ]:
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [38]:
os.getcwd()

'/Users/mark/Documents/flatiron/projects/so2/sulfur_dioxide_global'

In [39]:
os.listdir(os.getcwd())

['EDA.ipynb',
 'one.h5',
 '.DS_Store',
 'omps_readme.pdf',
 'README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'data']

## TIME issue:
- 80 seconds per h5 for 38k h5s gives us 844 hours of runtime for the function.
    - that is absolutely uncool
- is it an unavoidable issue with the amount of data in terms of functional calls?
    - i'm dealing with h5.Datasets, which are basically arrays
        - i have a feeling there's a faster way to array->dataframe than nested loop appending
    

In [42]:
len(files)

38976

In [64]:
sci.keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

### matrix to DataFrame directly

In [66]:
mat = sci['ColumnAmountSO2_PBL']
mat.shape

(320, 36)

In [ ]:
pd.DataFrame()

In [69]:
m = pd.DataFrame(mat) # 320 times a day, 36 cross-track positions are measured
m # could we just compress the 2d matrix to 1d, keeping track every 36?

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-4.029547e-01,9.849648e-01,2.042481e-01,4.230238e-01,2.397932e-01,1.811823e-01,2.592041e-01,5.033290e-01,2.876406e-01,2.669230e-01,...,4.640397e-01,4.937508e-02,3.414404e-01,7.708734e-02,1.848868e-01,-9.566254e-03,-1.734747e-01,1.331405e+00,-6.412577e-02,7.685578e-01
1,5.941088e-01,1.994952e-01,2.601840e-01,-3.926543e-01,4.075236e-01,5.000751e-01,-1.440047e-01,-8.859561e-03,-3.318212e-02,-4.084798e-01,...,4.130578e-02,1.336854e+00,6.432132e-02,-9.903105e-02,-3.579566e-01,-2.172945e-02,1.041501e-01,2.704256e-01,1.046404e-01,-7.228448e-02
2,1.646080e-01,-1.204964e-01,-1.745902e-01,-1.018621e-01,6.856797e-01,3.116942e-01,4.405132e-01,-1.187112e-01,-6.027973e-02,3.566566e-01,...,1.352778e-02,8.329485e-02,3.949238e-03,5.537626e-02,-1.838077e-02,4.684832e-01,-2.047891e-01,-1.427619e-01,6.267340e-01,4.096892e-02
3,7.128267e-02,8.145315e-02,9.390183e-02,-1.174992e-04,2.583468e-01,-1.184929e-01,2.506859e-02,2.105715e-01,2.007283e-01,2.295893e-01,...,-1.128365e-01,7.414775e-02,4.009604e-03,2.941097e-02,6.962321e-02,-1.348442e-02,9.868404e-02,7.643973e-01,5.092659e-01,1.269430e-01
4,1.892708e-01,-3.616448e-01,-2.538711e-01,-4.779992e-01,1.049691e-01,-1.959032e-01,-8.926398e-02,3.644142e-02,-3.869241e-01,-4.080417e-01,...,7.132534e-02,-6.885639e-02,-1.600974e-01,5.435780e-02,-5.518730e-03,6.408185e-02,-3.551501e-02,2.999925e-02,-2.185591e-02,1.799225e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,...,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30
316,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,...,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30
317,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,...,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30
318,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,...,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30,-1.267651e+30


- i have 9 matrices, each 320x36 for 11520 observations
    - or rather, ~4 are just 320 ( but i could extend them into 320x36 )
    - how can i make a dataframe shaped (11520, 9)
    - pretty sure pd.concat() will work here
- i think flattening it out to 1d could simplify things greatly
    - is there any reason to keep 2d shape? no...?
        - from a data structure standpoint, the 2d shape tells us:
            - "which observations belong to the same track-position?"
        - so if we flatten all matrices equally (and for the 1d, just multiply by 36)
            - then the structure should be solved / positions preserved?
        - then if i have 9 (11520) matrices, i basically have a dataframe

In [86]:
matrix = mat.value # convert to nparray
matrix

array([[-4.02954668e-01,  9.84964788e-01,  2.04248086e-01, ...,
         1.33140492e+00, -6.41257688e-02,  7.68557787e-01],
       [ 5.94108820e-01,  1.99495152e-01,  2.60183960e-01, ...,
         2.70425618e-01,  1.04640394e-01, -7.22844824e-02],
       [ 1.64608046e-01, -1.20496437e-01, -1.74590215e-01, ...,
        -1.42761916e-01,  6.26734018e-01,  4.09689248e-02],
       ...,
       [-1.26765060e+30, -1.26765060e+30, -1.26765060e+30, ...,
        -1.26765060e+30, -1.26765060e+30, -1.26765060e+30],
       [-1.26765060e+30, -1.26765060e+30, -1.26765060e+30, ...,
        -1.26765060e+30, -1.26765060e+30, -1.26765060e+30],
       [-1.26765060e+30, -1.26765060e+30, -1.26765060e+30, ...,
        -1.26765060e+30, -1.26765060e+30, -1.26765060e+30]], dtype=float32)

In [87]:
matrix.shape

(320, 36)

In [92]:
pbl = matrix.ravel() # looking at the 1st and last values, it seems like order is preserved (why wouldn't it be?)
pbl

array([-4.0295467e-01,  9.8496479e-01,  2.0424809e-01, ...,
       -1.2676506e+30, -1.2676506e+30, -1.2676506e+30], dtype=float32)

In [93]:
pbl.shape

(11520,)

In [94]:
sci.keys()

<KeysViewHDF5 ['CloudFraction', 'ColumnAmountO3', 'ColumnAmountSO2_PBL', 'ColumnAmountSO2_STL', 'ColumnAmountSO2_TRL', 'ColumnAmountSO2_TRM', 'ColumnAmountSO2_TRU', 'FittingWindow_STL', 'FittingWindow_TRL', 'FittingWindow_TRM', 'FittingWindow_TRU', 'Flag_SAA', 'Flag_SO2', 'RadiativeCloudFraction', 'Reflectivity331', 'SLER', 'UVAerosolIndex', 'Wavelengths_SLER', 'dNdR', 'nPrincipalComponents']>

In [95]:
clouds = sci['RadiativeCloudFraction'].value.ravel()

In [96]:
clouds.shape

(11520,)

In [97]:
clouds_lst = list(clouds)
pbl_lst = list(pbl)
df = pd.DataFrame(list(zip(clouds_lst, pbl_lst)), columns=['clouds', 'pbl'])
df

,clouds,pbl
0,2.995895e-01,-4.029547e-01
1,0.000000e+00,9.849648e-01
2,0.000000e+00,2.042481e-01
3,5.429736e-01,4.230238e-01
4,7.288764e-01,2.397932e-01
...,...,...
11515,-1.267651e+30,-1.267651e+30
11516,-1.267651e+30,-1.267651e+30
11517,-1.267651e+30,-1.267651e+30
11518,-1.267651e+30,-1.267651e+30


In [100]:
df2 = df.copy() # deep


In [105]:
print(df.shape, df2.shape)

(11520, 2) (11520, 2)


In [104]:
d3 = pd.concat([df,df2])
d3

,clouds,pbl
0,2.995895e-01,-4.029547e-01
1,0.000000e+00,9.849648e-01
2,0.000000e+00,2.042481e-01
3,5.429736e-01,4.230238e-01
4,7.288764e-01,2.397932e-01
...,...,...
11515,-1.267651e+30,-1.267651e+30
11516,-1.267651e+30,-1.267651e+30
11517,-1.267651e+30,-1.267651e+30
11518,-1.267651e+30,-1.267651e+30


### nice. i can just flatten the arrays and dataframe it
- this problem was tough until i let go of the notion that i need the 320x36 shape
    - when you're stuck, reconsidering your assumptions/accepted 'requirements' can help
- so the new process_h5_matrix() function:
    - open h5
    - assign datasets to vars
    - if 2d: flatten to 1d
    - if 1d: multiply length by 36
    - list & zip into DF

In [144]:
df = create_base_df()
start = time.time()
df = process_orbit(df, filename)
end = time.time()
print(f'time:  {end-start}')

11521
time:  0.029990196228027344


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [132]:
df.shape

(321, 10)

## matrix test: much faster, but warning
time:  0.01427316665649414
/Users/mark/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


## good to remember: work with matrices when you can, it's much faster than iterating i,j 

In [122]:
filename

'one.h5'

### now build the process_all_orbits()

In [152]:
!pip install tqdm

In [169]:
st = f'{1}s'
st

'1s'

In [173]:
def create_base_df():
    d = {'lat':[0.0],'long':[0.0],'sat_lat':[0.0],'sat_long':[0.0],'sat_alt':[0.0],'time':[0.0],'sza':[0.0],
      'pbl':[0.0],'anom':[0],'cloud':[0.0]}
    df = pd.DataFrame(data=d)
    return df
    

def process_orbit(df, h5): # pandas DF, filename string. adds h5's observations to dataframe
    f = h5py.File(h5, 'r') # read file
    geo = f['GEOLOCATION_DATA'] # h5 Groups architecture is similar to dict
    sci = f['SCIENCE_DATA']
    # hdf.Datasets -> np.array -> list
    lat = list(geo['Latitude'].value.ravel())
    long = list(geo['Longitude'].value.ravel()) # is there a less verbose way to do this?
    sat_lat = list(geo['SpacecraftLatitude'].value.ravel())*36 # extend the 1d arrays
    sat_long = list(geo['SpacecraftLongitude'].value.ravel())*36 # don't forget to ravel like i did
    sat_alt = list(geo['SpacecraftAltitude'].value.ravel())*36
    time = list(geo['TimeUTC'].value.ravel())*36 # 36 measurements per position means one "time" value for every consecutive 36 measurements
    sza = list(geo['SolarZenithAngle'].value.ravel())
    pbl = list(sci['ColumnAmountSO2_PBL'].value.ravel())
    anom = list(sci['Flag_SAA'].value.ravel())
    # combine lists into df
    new = pd.DataFrame(list(zip(lat, long, sat_lat, sat_long, sat_alt, time, sza, pbl, anom)),
                       columns=["lat", "long", "sat_lat", "sat_long", 'sat_alt', "time", "sza", "pbl", "anom"])
    # we don't have to filter out by SZA, Anomaly etc quite yet. save that for after the data's gathered
    df = pd.concat([df, new]) # merge 11,520 into existing df
#     print(len(df))
    return df

In [174]:
df = create_base_df()

In [176]:
from tqdm import tqdm
# def process_all_h5(df): # loads all h5s into a dataframe. will save locally every ~500 files
#     mypath = os.getcwd() + "/data/" # get all files in data folder
#     files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#     for f in tqdm(files): # h5py is kind of picky with absolute filenames & root folder files
#         path = mypath+f
#         df = process_orbit(df, path) # ah, that did the trick
        # enumerate instead of tqdm to allow saving every 100 .h5s
    # more stable in case of crash. could also write this to upload to mysql & keep track of processed/unprocessed files
def process_all_h5(df): # loads all h5s into a dataframe. will save locally every ~500 files
    mypath = os.getcwd() + "/data/" # get all files in data folder
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    for c,f in enumerate(files): # h5py is kind of picky with absolute filenames & root folder files
        print(c)
        path = mypath+f
        df = process_orbit(df, path) # ah, that did the trick
        if c/100 == 0:
            st = f'increment_{c}'
            df.to_csv(st+'.csv')
    return df
start = time.time()
df = process_all_h5(df)
df.to_csv('all_data.csv')
end = time.time()
print(f'time:   {end-start}')

0
1
2


/Users/mark/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


KeyboardInterrupt: 

In [168]:
2/100


0.02

In [166]:
df.to_csv('data.csv')

In [ ]:
lst = []
for filename in os.listdir(os.getcwd()):
    if filename.endswith('.h5'):
        with h5py.File(filename, 'r') as f:
            print('keys: %s' %f.keys())
            a_group_key = list(f.keys())[0]
            data = list(f[a_group_key])